# **Data Formatting Pipeline**

In [ ]:
!git clone https://github.com/OscarMoliina/betterlifebetterhealth.git

Cloning into 'betterlifebetterhealth'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (77/77), done.


#### Llegir dades i paquets necessaris:

In [ ]:
!pip3 install pyspark
import pyspark
from pyspark.sql import SparkSession
import duckdb

spark = SparkSession.builder\
    .appName("Parquet to DuckDB") \
    .master("local[*]") \
    .config("spark.jars", "/content/betterlifebetterhealth/src/utils/duckdb.jar") \
    .getOrCreate()

# Leer el archivo Parquet
demography_df = spark.read.parquet('/content/betterlifebetterhealth/data/parquet/demography.parquet')
mh_df = spark.read.parquet('/content/betterlifebetterhealth/data/parquet/mental_health.parquet')
society_df = spark.read.parquet('/content/betterlifebetterhealth/data/parquet/society.parquet')

### Save Basic

In [ ]:
d = {
    demography_df:"demography_df",
    mh_df:"mh_df",
    society_df:'society_df'
}

for k,v in d.items():
    k.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{v}.db") \
        .option("dbtable", f"{v}") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .save()

### Guardat db a Github:

In [ ]:
# Clone the repository
%cd betterlifebetterhealth/data/db

# Remove old files
!rm -rf demography_df.db
!rm -rf society_df.db
!rm -rf mh_df.db

# Copy new files from your local directory to this directory
!cp -R /content/*.db .

/content/betterlifebetterhealth/data/db


In [ ]:
#CODI A LA DOCUMENTACIO NO POSAT AQUÍ PER PRIVACITAT DE USUARI